In [4]:
# 모션감지 cctv
import cv2
import numpy as np
# 카메라 감도 설정
thresh = 25 # 픽셀값 기준치 설정
max_diff = 5 #  픽셀 수 기준치  설정

In [6]:
# 카메라 캡션 
a,b,c = None,None,None
cap =  cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,480)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,320)

if cap.isOpened():
    ret,a = cap.read()
    ret,b = cap.read()
    while ret:
        ret,c =  cap.read()
        draw = c.copy()    # 출력영상에 사용할 복제본
        if not ret:
            break
        
        # 3개의 영상을 그레이 스케일로 변경
        a_gray = cv2.cvtColor(a, cv2.COLOR_RGB2GRAY)
        b_gray = cv2.cvtColor(b, cv2.COLOR_RGB2GRAY)
        c_gray = cv2.cvtColor(c, cv2.COLOR_RGB2GRAY)
        
        # a-b, b-c 절대값의 차
        diff1 = cv2.absdiff(a_gray, b_gray)
        diff2 = cv2.absdiff(b_gray, c_gray)
        
        # 스레드홀드로 기준치 이내의 차이는 무시
        ret, diff1_t = cv2.threshold(diff1,thresh,255,cv2.THRESH_BINARY)
        ret, diff2_t = cv2.threshold(diff2,thresh,255,cv2.THRESH_BINARY)
        
        # 두개의 영상의 차이가 모두 발견된 경우
        diff = cv2.bitwise_and(diff1_t, diff2_t)
        
        #노이즈 제거( 영상끼리 연산을 한 경우 노이즈 생긴다)
        k = cv2.getStructuringElement(cv2.MORPH_CROSS, (3,3))
        diff = cv2.morphologyEx(diff, cv2.MORPH_OPEN,k)
        
        #움직임을 감지한 영역에 사각형
        diff_cnt = cv2.countNonZero(diff)
        if diff_cnt > max_diff:
            nzero = np.nonzero(diff) # 0이 아닌 픽셀의 좌표
            cv2.rectangle(draw, (min(nzero[1]), min(nzero[0])), (max(nzero[1]), max(nzero[0])),
                         (0,255,0), 2 )
            cv2.putText(draw, 'Motion Detected', (10,10), cv2.FONT_HERSHEY_DUPLEX, 0.5, (0,0,255) )
        
        stacked = np.hstack((draw, cv2.cvtColor(diff, cv2.COLOR_GRAY2BGR)))
        cv2.imshow('motion sensor', stacked)
        
        a = b
        b = c
        
        if cv2.waitKey(1) & 0xFF == 27:
            break